<div align=right>
Winter 2025<br>
Nardin<br>
HW2<br>
Yolanda<br>
xpan02<br>
</div>


<font color='darkblue'> <h2 align=center> Homework 2: Web Scraping</h2> </font>

**Instructions** 

Please complete the two tasks described below. Add as many code blocks as you'd like. 

Submit on Gradescope your Jupyter notebook with the homework completed and the output of your code visible. Complete both tasks.

HWs must be completed individually and are graded on a Pass-Fail basis. Before submitting, see the Syllabus for homework policies (what qualifies as a "Pass", deadlines, regrading, etc.)

At the end of the notebook, list all resources you consulted (e.g., if you consult specific websites, please add all links to them; if you use AI, explain for which task you used it and how). Do not submit code you do not fully understand or you have not written yourself.

**Task 1:**

In class, we learned several ways to approach web scraping, such as using HTML with specific attributes, using HTML only, using helper tools like "SelectorGadget", and scraping with XPath.

For this task, select a website of your choice (e.g. a University of Chicago department, another university department, a Wikipedia page, or any other website that interests you). Use at least two of the four approaches discussed in class to scrape something meaningful from the website. You are free to choose what you scrape, but your scraper should involve at least a few lines of code (for reference, you can use the in-class example with the University of Arizona faculty members: your scraping project should have a similar or higher level of complexity).

*Reflections*: After completing the code, include a Markdown block with a short reflection about your experience. We want to know what you have learned from this excercise, which approach worked best for you and why, how you plan to approach future web scraping tasks based on this experience, and anything else you would like to share.

In [55]:
# your code here; add as many code chunks as you like

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

macss_prof = 'https://macss.uchicago.edu/people/macss-professors'
response = requests.get(macss_prof)

# user-agent
header = {"User-Agent": "for uchicago scraping homework"}
response = requests.get(macss_prof, headers = header)

print("Our response code is:", response.status_code)

Our response code is: 200


In [57]:
response_txt = response.text
soup = bs(response_txt, 'html.parser')
# print(soup.prettify)
print(soup.find("title"))

<title>MACSS Professors | Masters in Computational Social Science</title>


In [58]:
# Use the Tag with its Unique Attribute
contacts = []

for row in soup.find_all('div', attrs = {'class': 'views-row'}):

    # scrape names
    name_tag = row.find('a', attrs = {'href': True})
    if name_tag:
        name = name_tag.text.strip()
    else:
        name = "NA"
    
    # scrape titles
    title_tag = row.find('div', attrs = {'class': 'bio-subtitle'})
    if title_tag:
        title = title_tag.text.strip()
    else:
        title = "NA"
    
    # scrape emails
    emails_tag = row.find('div', attrs = {'class': 'bio-email'})
    if emails_tag:
        email = emails_tag.text.strip()
    else:
        email = "NA"
    
    # scrape phones
    phone_tag = row.find('div', attrs = {'class': 'bio-phone'})
    if phone_tag:
        phone = phone_tag.text.strip()
    else:
        phone = "NA"
    
    contacts.append({'Name': name,
                     'Title': title,
                     'Email': email,
                     'Phone': phone
                     })

contacts


[{'Name': 'Marc Berman',
  'Title': 'Chair and Professor, Department of Psychology',
  'Email': 'bermanm@uchicago.edu',
  'Phone': 'Phone: (773) 702-1436'},
 {'Name': 'Jon Clindaniel',
  'Title': 'Associate Director of Undergraduate Studies and Assistant Senior Instructional Professor',
  'Email': 'jclindaniel@uchicago.edu',
  'Phone': 'Phone: (773) 702-4829'},
 {'Name': 'Jean Clipperton',
  'Title': 'Associate Director of MACSS; Associate Senior Instructional Professor',
  'Email': 'clipperton@uchicago.edu',
  'Phone': 'NA'},
 {'Name': 'Henry K. Dambanemuya',
  'Title': 'Assistant Instructional Professor',
  'Email': 'hdambane@uchicago.edu',
  'Phone': 'NA'},
 {'Name': 'James Evans',
  'Title': 'Max Palevsky Professor, Department of Sociology; Faculty Director, Masters in Computational Social Science Program',
  'Email': 'jevans@uchicago.edu',
  'Phone': 'Phone: (773) 702-9168'},
 {'Name': 'Min Sok Lee',
  'Title': 'Assistant Senior Instructional Professor in Economics, Director of MA

In [59]:
# Tag Selector
contacts_gadget = []

for row in soup.find_all('div', attrs = {'class': 'views-row'}): 
    
    name_tag = row.find('h2', attrs = {'class': 'no-tags'})
    name = name_tag.text.replace("\n", " ") if name_tag else "NA"

    title_tag = row.find('div', attrs = {'class': 'bio-subtitle'})
    title = title_tag.text.replace("\n", " ").strip() if title_tag else "NA"
      
    email_tag = row.find('div', attrs = {'class': 'bio-email'})
    email = email_tag.text if emails_tag else "NA"

    phone_tag = row.find('div', attrs = {'class': 'bio-phone'})
    phone = phone_tag.text.replace("\n", " ").strip() if phone_tag else "NA"

    contacts_gadget.append([name, title, email, phone])  

for row in contacts:
    print (row)

{'Name': 'Marc Berman', 'Title': 'Chair and Professor, Department of Psychology', 'Email': 'bermanm@uchicago.edu', 'Phone': 'Phone: (773) 702-1436'}
{'Name': 'Jon Clindaniel', 'Title': 'Associate Director of Undergraduate Studies and Assistant Senior Instructional Professor', 'Email': 'jclindaniel@uchicago.edu', 'Phone': 'Phone: (773) 702-4829'}
{'Name': 'Jean Clipperton', 'Title': 'Associate Director of MACSS; Associate Senior Instructional Professor', 'Email': 'clipperton@uchicago.edu', 'Phone': 'NA'}
{'Name': 'Henry K. Dambanemuya', 'Title': 'Assistant Instructional Professor', 'Email': 'hdambane@uchicago.edu', 'Phone': 'NA'}
{'Name': 'James Evans', 'Title': 'Max Palevsky Professor, Department of Sociology; Faculty Director, Masters in Computational Social Science Program', 'Email': 'jevans@uchicago.edu', 'Phone': 'Phone: (773) 702-9168'}
{'Name': 'Min Sok Lee', 'Title': 'Assistant Senior Instructional Professor in Economics, Director of MA Programs in Economics', 'Email': 'mslee@uc

In [60]:
# lxml
from lxml import html

tree = html.fromstring(str(soup))

faculty_blocks = tree.xpath('//div[@class="block-text"]')
contacts_lxml = []

for block in faculty_blocks:
    name = block.xpath('.//h2/a/text()')
    title = block.xpath('.//div[@class="bio-subtitle"]/text()')
    email = block.xpath('.//div[@class="bio-email"]/text()')
    phone = block.xpath('.//div[@class="bio-phone"]/text()')
    
    prof = {
        'Name': name[0].strip() if name else "NA",
        'Title': title[0].strip() if title else "NA",
        'Email': email[0].strip() if email else "NA",
        'Phone': phone[0].strip() if phone else "NA"
    }
    contacts_lxml.append(prof)

contacts_lxml

[{'Name': 'Marc Berman',
  'Title': 'Chair and Professor, Department of Psychology',
  'Email': 'NA',
  'Phone': '(773) 702-1436'},
 {'Name': 'Jon Clindaniel',
  'Title': 'Associate Director of Undergraduate Studies and Assistant Senior Instructional Professor',
  'Email': 'NA',
  'Phone': '(773) 702-4829'},
 {'Name': 'Jean Clipperton',
  'Title': 'Associate Director of MACSS; Associate Senior Instructional Professor',
  'Email': 'NA',
  'Phone': 'NA'},
 {'Name': 'Henry K. Dambanemuya',
  'Title': 'Assistant Instructional Professor',
  'Email': 'NA',
  'Phone': 'NA'},
 {'Name': 'James Evans',
  'Title': 'Max Palevsky Professor, Department of Sociology; Faculty Director, Masters in Computational Social Science Program',
  'Email': 'NA',
  'Phone': '(773) 702-9168'},
 {'Name': 'Min Sok Lee',
  'Title': 'Assistant Senior Instructional Professor in Economics, Director of MA Programs in Economics',
  'Email': 'NA',
  'Phone': '(773) 834-1754'},
 {'Name': 'Victor O. Lima',
  'Title': 'Senior

In [64]:
#  better format/visualization

df = pd.DataFrame(contacts)
display(df)

# export csv
df.to_csv(r'/Users/yolandapan/Library/CloudStorage/OneDrive-TheUniversityofChicago/30122_Nardin_Data/macss_prof.csv', encoding = 'utf-8', index = False)
print('Data exported!')

,Name,Title,Email,Phone
0,Marc Berman,"Chair and Professor, Department of Psychology",bermanm@uchicago.edu,Phone: (773) 702-1436
1,Jon Clindaniel,Associate Director of Undergraduate Studies an...,jclindaniel@uchicago.edu,Phone: (773) 702-4829
2,Jean Clipperton,Associate Director of MACSS; Associate Senior ...,clipperton@uchicago.edu,NA
3,Henry K. Dambanemuya,Assistant Instructional Professor,hdambane@uchicago.edu,NA
4,James Evans,"Max Palevsky Professor, Department of Sociolog...",jevans@uchicago.edu,Phone: (773) 702-9168
5,Min Sok Lee,Assistant Senior Instructional Professor in Ec...,mslee@uchicago.edu,Phone: (773) 834-1754
6,Victor O. Lima,Senior Instructional Professor in Economics; C...,vlima@midway.uchicago.edu,Phone: (773) 834-6672
7,Sabrina Nardin,Assistant Instructional Professor,nardin@uchicago.edu,NA
8,David A. Peterson,Assistant Instructional Professor,dapeterson@uchicago.edu,NA
9,Ali Sanaei,Associate Instructional Professor,sanaei@uchicago.edu,Phone: (773) 702-3474


Data exported!


Task 1 reflections:  please replace this sentence with your reflections (see task description for details)

Finding the clear path as reference is very important in scraping. I tried three approaches of tag, selectorgadget, and XPaths. I think tag and XPath have their own advantages as one with require simpler path and the other is very clear. For the simple web I scarped, XPath is super helpful, but I would prefer using tag for future scarpings. I do not like the selector gadget because it was not as clear and intuitive as expected. 

<b>Task 2:</b>

In class, we wrote code to collect all the URLs from a single webpage and retrieve movie information (e.g., title, plot, etc.) from each of these URLs. Your task is to extend this code so that it collects movie information from multiple pages, not just one.

Requirements for Your Scraper:
* Your scraper must collect all the required information (title, plot, script), accounting for missing data, from a random sample of 5 pages. Although the total number of pages is ~1794, do not scrape all of them! Scraping 5 pages is more than sufficient for this homework.
* Your scraper must pause between requests to avoid overloading the server: use `time.sleep()` to include a delay between requests to ensure your scraper doesn’t go too fast
* You may reuse the in-class code with minor adjustment (e.g., you need to implement code for turning pages, etc.) or rewrite the scraper in your own style. Regardless of your approach write your code in functions and each function must include docstrings.

CHECK THIS: For example, include functions to: generate a list of pages to scrape; make requests and retrieve HTML data; Parse the HTML and extract the relevant information, etc.

*Reflections*: After completing the code, include a Markdown block with a short reflection about your experience. We want to know what you have learned from this excercise, what is still unclear, or anything else you would like to share.

In [65]:
# your code here; add as many code chunks as you like

In [40]:
import time
import random

In [41]:
def request_retrieve_parse_data(url):
    '''
       Retrieves and parses webpage content using BeautifulSoup.
       
       Input:
       url (str): Target webpage URL
       
       Returns:
       bs4.BeautifulSoup: Parsed HTML content
    '''
    header = { "User-Agent" : "small scraper for classroom purposes xpan02@uchicago.edu" } 
    response = requests.get(url, headers = header)
    time.sleep(random.randint(2, 4))
    content = response.text
    soup = bs(content, 'html.parser')
    return soup

In [47]:
def get_movie_links(page_url):
    """
    Extract all movie links from a given page URL

    Input:
    page_url (str): URL of the movie listing page

    Returns:
    list: List of complete movie URLs
    """
    links = []
    base_url = 'https://subslikescript.com'
    soup = request_retrieve_parse_data(page_url)
    for element in soup.find_all('ul', class_= 'scripts-list'): # ul is the tag, class its attribute
        for link in element.find_all('a', href = True):  # a is the tag, href its attribute
            links.append(link['href'])
    full_urls = [(base_url + link) for link in links]
    return full_urls

In [48]:
def extract_info(url):
   '''
   Extracts title, plot and script from a movie webpage.
   
   Inputs:
   url (str): URL of the movie webpage

   Returns:
   list: [title, plot, script] where each is a string or "NA" if not found
   '''
   soup = request_retrieve_parse_data(url)
   title = soup.find('h1').get_text() if soup.find('h1') else "NA"
   plot = soup.find('p', class_ = 'plot').get_text() if soup.find('p', class_ = 'plot') else "NA"
   script = soup.find('div', id = 'cue-app', class_ = 'full-script').get_text() if soup.find('div', class_ = 'full-script') else "NA"
   return [title, plot, script]

In [53]:
# empty list to store all responses
def collect_movie_data(): 
    '''
    Scrapes movie information from multiple random pages and stores in DataFrame.
    
    Returns:
    pandas.DataFrame: Contains columns ['title', 'plot', 'script'] with movie data
    '''
    requested_pages = []
    movies = pd.DataFrame(columns=['title', 'plot', 'script'])
    for _ in range(5):
        # Randomly selects 5 pages between 1-1994
        page_num = random.randint(1, 1994)
        url = f'https://subslikescript.com/movies?page={page_num}'
        # Extracts movie links from each page
        links = get_movie_links(url)
        for link in links:
            if link not in requested_pages:
                requested_pages.append(link)
                # Scrapes title, plot and script for each unique movie
                info = extract_info(link)    
                # Stores data in pandas DataFrame
                movies.loc[len(movies)] = info

    return movies


In [54]:
if __name__ == "__main__":
   movies_df = collect_movie_data() 
   display(movies_df)

,title,plot,script
0,The Edge of the Garden (2011) - full transcript,Brian Connor works as an executive in his fami...,"\n\n[♪♪♪]\n\n\nOH, MY GOD, WHAT TIME IS IT?\n\..."
1,This Property Is Condemned (1966) - full trans...,"A railroad official, Owen Legate comes to Dods...",\n\nWish me red roses\n\n\nAnd yellow balloons...
2,The Gentleman Driver (2018) - full transcript,The Gentleman Driver gives an inside look into...,"\n\nBut, ultimately, the piece of humanity\nth..."
3,Night and the City (1950) - full transcript,Harry Fabian is a London hustler with ambitiou...,\n\n'Night and the city.\n\n\n'The night is to...
4,Agatha and the Truth of Murder (2018) - full t...,"In 1926, with her personal life in tatters and...","\n\n[""What a friend we have\nin Jesus"" hymn pl..."
...,...,...,...
145,Yogi Bear's All-Star Comedy Christmas Caper (1...,"Dear Jellystone Visitors, Happy holidays from ...",\n\n[engine revving]\n\n\n[no audio]\n\n\n[no ...
146,The Toy Gun (2016) - full transcript,THE TOY GUN is a funny dark comedy about a cha...,\n\n- The wine makes your\nlips look even redd...
147,Wish for Christmas (2016) - full transcript,When a high school senior finds out from her p...,"\n\n- Good morning, everyone.\n- Good morning...."
148,Woman's World (1954) - full transcript,Needing to fill the position of general manage...,\n\n♪ It's a woman's world.\n\n\n♪ When she's ...


Task 2 reflections: please replace this sentence with your reflections (see task description for details)

This task if more completed than the first one. Using pandasn be really useful in formatting and layouts. My key takeaway is the process or streamline of scraping:
- Setting up url -> requests -> parse (beautifulsoup, etc.)  
- create a list for storage (links/requested pages/souped pages/thing of interests)
- for loop + ***functions***
- remember to clean up the url to make sure it's complete

***Functions*** mentioned are as followings:
- generate a list of pages to scrape
    - *soup.find_all*
- make requests and retrieve HTML data
    - *requested.get.text*
- Parse the HTML and extract the relevant information 
    - *bs(response_txt, 'html.parser)*
- get the text of interests 
    - *response_text.find().get_text()*

**Resources:**
    
Add here all the resources you consulted to complete this homework (see homework instructions)

https://macss.uchicago.edu/people/macss-professors